In [1]:
import pandas as pd
from pathlib import Path
import gc
import pyarrow as pa
import pyarrow.parquet as pq

In [10]:
TRAIN_CSV = Path("../credit_default_prediction/input_data/train_data.csv")
TRAIN_LABELS_CSV = Path(
    "../credit_default_prediction//input_data/train_labels.csv")
TEST_CSV = Path('../credit_default_prediction//input_data/test_data.csv')
CHUNKSIZE = 15000

**Описание задачи**

American Express — глобальная интегрированная платежная компания. Являясь крупнейшим эмитентом платежных карт в мире, они предоставляют клиентам доступ к продуктам, информации и опыту, которые обогащают жизнь и способствуют успеху в бизнесе.

Целью этой задачи является прогнозирование вероятности того, что клиент не выплатит сумму остатка по кредитной карте в будущем, на основе его ежемесячного профиля клиента.

Набор данных содержит агрегированные характеристики профиля для каждого клиента на каждую дату выписки. Функции анонимизированы и нормализованы и делятся на следующие общие категории:

D_* = переменные просроченной задолженности
S_* = Расходные переменные
P_* = Платежные переменные
B_* = Балансовые переменные
R_* = Переменные риска
Следующие признаки, являются категоричными:

['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']

Задача состоит в том, чтобы предсказать для каждого customer_ID вероятность невыполнения платежа в будущем (target = 1).

Весь набор данных весит около 50 ГБ, поэтому просто прочитать их с помощью Pandas не выйдет. И требуется предварительно уменьшить размер данных.

In [3]:
df = pd.read_csv(TRAIN_CSV, nrows=10)

In [4]:
df.head()

,customer_ID,S_2,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,...,D_136,D_137,D_138,D_139,D_140,D_141,D_142,D_143,D_144,D_145
0,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-03-09,0.938469,0.001733,0.008724,1.006838,0.009228,0.124035,0.008771,0.004709,...,NaN,NaN,NaN,0.002427,0.003706,0.003818,NaN,0.000569,0.000610,0.002674
1,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-04-07,0.936665,0.005775,0.004923,1.000653,0.006151,0.126750,0.000798,0.002714,...,NaN,NaN,NaN,0.003954,0.003167,0.005032,NaN,0.009576,0.005492,0.009217
2,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-05-28,0.954180,0.091505,0.021655,1.009672,0.006815,0.123977,0.007598,0.009423,...,NaN,NaN,NaN,0.003269,0.007329,0.000427,NaN,0.003429,0.006986,0.002603
3,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-06-13,0.960384,0.002455,0.013683,1.002700,0.001373,0.117169,0.000685,0.005531,...,NaN,NaN,NaN,0.006117,0.004516,0.003200,NaN,0.008419,0.006527,0.009600
4,0000099d6bd597052cdcda90ffabf56573fe9d7c79be5f...,2017-07-16,0.947248,0.002483,0.015193,1.000727,0.007605,0.117325,0.004653,0.009312,...,NaN,NaN,NaN,0.003671,0.004946,0.008889,NaN,0.001670,0.008126,0.009827


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Columns: 190 entries, customer_ID to D_145
dtypes: float64(185), int64(1), object(4)
memory usage: 15.0+ KB


При предварительном осмотре данных видно:
- customer_ID хранится в виде кэша и занимает много места
- числовые данные с типом float64(185) можно уменьшить заменим на float32
- S2 - это дата, требует преобразования в соответствующий тип
- а также можно перевести категориальные данные, которые нам известны из описания датасета, в тип category

In [17]:
float_vars = df.select_dtypes('float64').columns

In [18]:
cat_features = ['B_30', 'B_38', 'D_114', 'D_116', 'D_117',
                'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']

In [31]:
def prepair(input_path, output_path, chunksize=CHUNKSIZE):
    """
    Уменьшение размера датасета за счёт смены типа данных с 64 бит на 32,
    конвертации в parquet и других преобразований
    params:
    - input_path: str
    - output_path: str
    - chunksize: integer
    return: None
    """

    pq_writer = None

    for idx, chunk in enumerate(pd.read_csv(input_path, chunksize=chunksize)):
        print(f"id: {idx} Chunk size {chunk.shape}")

        chunk[float_vars] = chunk[float_vars].astype('float32')

        chunk['customer_ID'] = chunk['customer_ID'].str[-16:].apply(
            int, base=16).astype('int64')

        chunk[cat_features] = chunk[cat_features].astype('category')

        chunk['S_2'] = pd.to_datetime(chunk['S_2'])

        table = pa.Table.from_pandas(chunk)
        if idx == 0:
            pq_writer = pq.ParquetWriter(
                output_path, table.schema, compression='snappy')

        pq_writer.write_table(table)

        del chunk
        del table
        gc.collect()

    if pq_writer:
        pq_writer.close()
    gc.collect()

In [32]:
prepair(input_path=TRAIN_CSV, output_path='./train_data.parquet')

id: 0 Chunk size (15000, 190)
id: 1 Chunk size (15000, 190)
id: 2 Chunk size (15000, 190)
id: 3 Chunk size (15000, 190)
id: 4 Chunk size (15000, 190)
id: 5 Chunk size (15000, 190)
id: 6 Chunk size (15000, 190)
id: 7 Chunk size (15000, 190)
id: 8 Chunk size (15000, 190)
id: 9 Chunk size (15000, 190)
id: 10 Chunk size (15000, 190)
id: 11 Chunk size (15000, 190)
id: 12 Chunk size (15000, 190)
id: 13 Chunk size (15000, 190)
id: 14 Chunk size (15000, 190)
id: 15 Chunk size (15000, 190)
id: 16 Chunk size (15000, 190)
id: 17 Chunk size (15000, 190)
id: 18 Chunk size (15000, 190)
id: 19 Chunk size (15000, 190)
id: 20 Chunk size (15000, 190)
id: 21 Chunk size (15000, 190)
id: 22 Chunk size (15000, 190)
id: 23 Chunk size (15000, 190)
id: 24 Chunk size (15000, 190)
id: 25 Chunk size (15000, 190)
id: 26 Chunk size (15000, 190)
id: 27 Chunk size (15000, 190)
id: 28 Chunk size (15000, 190)
id: 29 Chunk size (15000, 190)
id: 30 Chunk size (15000, 190)
id: 31 Chunk size (15000, 190)
id: 32 Chunk size 

id: 260 Chunk size (15000, 190)
id: 261 Chunk size (15000, 190)
id: 262 Chunk size (15000, 190)
id: 263 Chunk size (15000, 190)
id: 264 Chunk size (15000, 190)
id: 265 Chunk size (15000, 190)
id: 266 Chunk size (15000, 190)
id: 267 Chunk size (15000, 190)
id: 268 Chunk size (15000, 190)
id: 269 Chunk size (15000, 190)
id: 270 Chunk size (15000, 190)
id: 271 Chunk size (15000, 190)
id: 272 Chunk size (15000, 190)
id: 273 Chunk size (15000, 190)
id: 274 Chunk size (15000, 190)
id: 275 Chunk size (15000, 190)
id: 276 Chunk size (15000, 190)
id: 277 Chunk size (15000, 190)
id: 278 Chunk size (15000, 190)
id: 279 Chunk size (15000, 190)
id: 280 Chunk size (15000, 190)
id: 281 Chunk size (15000, 190)
id: 282 Chunk size (15000, 190)
id: 283 Chunk size (15000, 190)
id: 284 Chunk size (15000, 190)
id: 285 Chunk size (15000, 190)
id: 286 Chunk size (15000, 190)
id: 287 Chunk size (15000, 190)
id: 288 Chunk size (15000, 190)
id: 289 Chunk size (15000, 190)
id: 290 Chunk size (15000, 190)
id: 291 

In [ ]:
prepair(input_path=TEST_CSV, output_path='./test_data.parquet')

In [ ]:
train_labels = pd.read_csv(TRAIN_LABELS_CSV)
train_labels['customer_ID'] = train_labels['customer_ID'].str[-16:].apply(
    int, base=16).astype('int64')
train_labels.to_parquet('./train_labels.parquet')